In [21]:
import pandas as pd
import datetime 
import pandasql as ps
import numpy as np

In [3]:
df = pd.read_csv('Clean Data/MBC1_all.csv')
df.head(5)

,ID,Sex,Time,Date,Condition,PA,TargetedSed,SatFat,FV,PA_outlier,...,SatFat_outlier,FV_outlier,educate,scwt,height,ethnicit,scrwaist,scrhip,age,L2wt
0,6,0,BL_01,6-Apr-05,2,570,270,0.0236,1,2,...,0,0,6,197,71,999999,999999.0,999999.0,999999,999.0
1,6,0,BL_02,7-Apr-05,2,750,229,0.1618,1,2,...,0,0,6,197,71,999999,999999.0,999999.0,999999,999.0
2,6,0,BL_03,8-Apr-05,2,0,120,0.0755,1,0,...,0,0,6,197,71,999999,999999.0,999999.0,999999,999.0
3,6,0,BL_04,9-Apr-05,2,105,165,0.0969,0,0,...,0,0,6,197,71,999999,999999.0,999999.0,999999,999.0
4,6,0,BL_05,10-Apr-05,2,0,225,0.1167,1,0,...,0,0,6,197,71,999999,999999.0,999999.0,999999,999.0


In [14]:
goal_list = list(df['Time'])
goal_abbr = []
for i in goal_list:
    if (i.startswith('Rx1')):
        goal_abbr.append('Rx1')
    elif(i.startswith('Rx23')):
        goal_abbr.append('Rx23')
    elif(i.startswith('FU')):
        goal_abbr.append(i[:3])
    elif(i.startswith('BL')):
        goal_abbr.append('BL') 

df['goal'] = goal_abbr
columns = ['ID', 'goal', 'Condition', 'PA', 'TargetedSed', 'SatFat', 'FV']
df_outcomes = df[columns]

In [16]:
df_sed = df[['goal', 'Condition', 'TargetedSed']]
df_sed = df_sed[df_sed.TargetedSed != 999999]
df_sed.groupby(by=['Condition','goal']).count().to_csv('check/1.csv')

df_pa_min = df[['goal', 'Condition', 'PA']]
df_pa_min = df_pa_min[df_pa_min.PA != 999999]
df_pa_min.groupby(by=['Condition','goal']).count().to_csv('check/2.csv')

df_fat = df[['goal', 'Condition', 'SatFat']]
df_fat = df_fat[df_fat.SatFat != 999999]
df_fat.groupby(by=['Condition','goal']).count().to_csv('check/3.csv')

df_fv = df[['goal', 'Condition', 'FV']]
df_fv = df_fv[df_fv.FV != 999999]
df_fv.groupby(by=['Condition','goal']).count().to_csv('check/4.csv')

In [19]:
df_1 = pd.read_csv('check/1.csv')
df_2 = pd.read_csv('check/2.csv')
df_3 = pd.read_csv('check/3.csv')
df_4 = pd.read_csv('check/4.csv')

#outter join all
df_all = pd.merge(df_1, df_2, how="outer", on=['Condition','goal'])
df_all = pd.merge(df_all, df_3, how="outer", on=['Condition','goal'])
df_all = pd.merge(df_all, df_4, how="outer", on=['Condition','goal'])

df_all.to_csv('check/count_condition_time.csv')

### Validation on record counts

In [39]:
df = pd.read_csv('Clean Data/MBC1_all.csv')
df.head(2)

,ID,Sex,Time,Date,Condition,PA,TargetedSed,SatFat,FV,PA_outlier,...,SatFat_outlier,FV_outlier,educate,scwt,height,ethnicit,scrwaist,scrhip,age,L2wt
0,6,0,BL_01,6-Apr-05,2,570,270,0.0236,1,2,...,0,0,6,197,71,999999,999999.0,999999.0,999999,999.0
1,6,0,BL_02,7-Apr-05,2,750,229,0.1618,1,2,...,0,0,6,197,71,999999,999999.0,999999.0,999999,999.0


In [40]:
df_base = df.loc[df['Time'].str.startswith('BL', na=False)]
df_Rx1 = df.loc[df['Time'].str.startswith('Rx1', na=False)]
df_Rx23 = df.loc[df['Time'].str.startswith('Rx23', na=False)]
df_FU = df.loc[df['Time'].str.startswith('FU', na=False)]

df_FU['Time_1'] = df_FU['Time'].astype('string').str[:3]

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
df_count_1 = ps.sqldf("select ID as study_id, Condition as condition, count(*) as count from df_base group by ID, Condition")
df_count_1['Time'] = ['baseline'] * df_count_1.shape[0]

df_count_2 = ps.sqldf("select ID as study_id, Condition as condition, count(*) as count from df_Rx1 group by ID, Condition")
df_count_2['Time'] = ['rx1'] * df_count_2.shape[0]

df_count_3 = ps.sqldf("select ID as study_id, Condition as condition, count(*) as count from df_Rx23 group by ID, Condition")
df_count_3['Time'] = ['rx23'] * df_count_3.shape[0]

df_count_4 = ps.sqldf("select ID as study_id, Time_1 as Time, Condition as condition, count(*) as count from df_FU group by ID, Condition, Time_1")

In [42]:
df_temp1 = df_count_1.append(df_count_2)
df_temp2 = df_temp1.append(df_count_3)
df_temp3 = df_temp2.append(df_count_4)
df_temp3.head(5)

,study_id,condition,count,Time
0,6,2,14,baseline
1,7,0,14,baseline
2,12,0,14,baseline
3,14,3,14,baseline
4,15,0,14,baseline


In [51]:
df_p = ps.sqldf("select study_id, condition, sum(count) as count from df_temp3 group by study_id, condition")

df_count = pd.pivot_table(df_p, values='count', index=['condition'],
                    columns=['study_id'], aggfunc=np.sum)

df_count = df_count.fillna(0)
#df_count.to_csv('check/temp/count.csv')
df_p

,study_id,condition,count
0,6,2,62
1,7,0,63
2,8,0,28
3,12,0,55
4,14,3,62
...,...,...,...
137,310,0,63
138,313,1,63
139,315,1,62
140,323,1,69


In [19]:
def Stats_1(df):
    if(df.shape[0] == 0):
        return([0,0,0,0,0])
    
    l = list(df['count'])
    Max = np.max(l)
    Min = np.min(l)
    Mean = np.mean(l)
    Median = np.median(l)
    STD = np.std(l)
    return([Mean, Median, STD, Min, Max])

In [32]:
# check statistically based on 4 conditions
cond0 = df_temp3[df_temp3['condition']==0]
cond1 = df_temp3[df_temp3['condition']==1]
cond2 = df_temp3[df_temp3['condition']==2]
cond3 = df_temp3[df_temp3['condition']==3]

cond_0 = []
for time in ['baseline','rx1','rx23','FU1','FU2','FU3','FU4','FU5','FU6','FU7','FU8']:
    df_temp = cond0[cond0['Time'] == time]
    cond_0.append(Stats_1(df_temp))
    
cond_1 = []
for time in ['baseline','rx1','rx23','FU1','FU2','FU3','FU4','FU5','FU6','FU7','FU8']:
    df_temp = cond1[cond1['Time'] == time]
    cond_1.append(Stats_1(df_temp))
    
cond_2 = []
for time in ['baseline','rx1','rx23','FU1','FU2','FU3','FU4','FU5','FU6','FU7','FU8']:
    df_temp = cond2[cond2['Time'] == time]
    cond_2.append(Stats_1(df_temp))
    
cond_3 = []
for time in ['baseline','rx1','rx23','FU1','FU2','FU3','FU4','FU5','FU6','FU7','FU8']:
    df_temp = cond3[cond3['Time'] == time]
    cond_3.append(Stats_1(df_temp))
    
df_0 = pd.DataFrame(cond_0)
# specifying cloumn names
df_0.columns = ['Mean', 'Median', 'STD', 'Min', 'Max']

df_2 = pd.DataFrame(cond_2)
# specifying cloumn names
df_2.columns = ['Mean', 'Median', 'STD', 'Min', 'Max']

df_3 = pd.DataFrame(cond_3)
  # specifying cloumn names
df_3.columns = ['Mean', 'Median', 'STD', 'Min', 'Max']

df_1 = pd.DataFrame(cond_1)
# specifying cloumn names
df_1.columns = ['Mean', 'Median', 'STD', 'Min', 'Max']

In [33]:
df_0.to_csv('check/temp/0.csv', index = False)
df_1.to_csv('check/temp/1.csv', index = False)
df_2.to_csv('check/temp/2.csv', index = False)
df_3.to_csv('check/temp/3.csv', index = False)